# Runnable
## Overview

LangChain 的 ```Runnable``` 物件提供了一種模組化和靈活的方法來設計工作流程，通過啟用鏈接、平行執行和數據轉換。這些工具允許高效處理結構化輸入和輸出，並減少代碼開銷。

Key Components is:

- **```RunnableLambda```**: 一個輕量級工具，通過 lambda 函數應用自定義邏輯，適用于動態和快速數據轉換。
- **```RunnablePassthrough```**: 設計用於傳遞未修改的輸入數據或在配對 ```assign()``` 方法時增強輸入數據。
- **```itemgetter```**: Python ```operator``` 模組工具，用於高效提取結構化數據（如字典或元組）中的特定鍵或索引。

這些工具可以組合使用來構建強大的工作流程，例如:

- 使用 ```itemgetter``` 提取和處理特定數據元素。
- 使用 ```RunnableLambda``` 執行自定義轉換。
- 使用 ```Runnable``` 鏈構建端到端數據管道。

透過這些組件，使用者可以設計可擴展和可重用的機器學習和數據處理工作流程。


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Efficient Data Handling with RunnablePassthrough](#efficient-data-handling-with-runnablepassthrough)
- [Efficient Parallel Execution with RunnableParallel](#efficient-parallel-execution-with-runnableparallel)
- [Dynamic Processing with RunnableLambda](#dynamic-processing-with-runnablelambda)
- [Extracting Specific Keys Using itemgetter](#extracting-specific-keys-using-itemgetter)

### References

- [LangChain Documentation: Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- [LangChain Documentation](https://python.langchain.com/docs/how_to/lcel_cheatsheet/)
- [Python operator module: itemgetter](https://docs.python.org/3/library/operator.html#operator.itemgetter)

---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

You can also load the ```OPEN_API_KEY``` from the ```.env``` file.

In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
# Set local environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "05-Runnable",
    }
)

Environment variables have been set successfully.


## Efficient Data Handling with RunnablePassthrough

```RunnablePassthrough``` 是一個輕量級工具，用於簡化數據處理工作流程，通過傳遞未修改的輸入數據或增強輸入數據。其靈活性使其成為處理數據管道中最小轉換或選擇性增強的理想工具。

1. **Simple Data Forwarding**

- 適合於不需要轉換的場景，例如記錄原始數據或將其傳遞到下游系統。

2. **Dynamic Data Augmentation**

- 用於向輸入數據添加元數據或上下文，以便在機器學習管道或分析系統中使用。

---
- ```RunnablePassthrough``` 可以傳遞未修改的輸入數據或附加鍵值。
- 當 ```RunnablePassthrough()``` 本身被調用時，它簡單地接受輸入並傳遞它。
- 當使用 ```RunnablePassthrough.assign(...)``` 調用時，它接受輸入並添加 assign 函數提供的附加參數。

### RunnablePassthrough

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Create the prompt and llm
prompt = PromptTemplate.from_template("What is 10 times {num}?")
llm = ChatOpenAI(temperature=0)

# Create the chain
chain = prompt | llm

當使用 ```invoke()``` 執行鏈式步驟時，輸入數據必須是字典類型。

In [7]:
# Execute the chain : input dtype as 'dictionary'
chain.invoke({"num": 5})

AIMessage(content='10 times 5 is equal to 50.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-420bc9dc-12eb-4f7a-a2c4-8e521b3d952d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

However, with the update to the LangChain library, if the template includes **only one variable**, it is also possible to pass just the value directly.

In [8]:
# Execute the chain : input value directly
chain.invoke(5)

AIMessage(content='10 times 5 is equal to 50.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3723d11b-89e1-490c-8946-b724fbc2c46d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

這裏是 ```RunnablePassthrough``` 的一個例子。
```RunnablePassthrough``` 是一個 ```runnable``` 物件，具有以下特點:

1. **Basic Operation**
   - 執行一個簡單的 pass-through 函數，將輸入值直接轉發到輸出
   - 可以獨立使用 ```invoke()``` 方法執行

2. **Use Cases**
   - 用於通過鏈式步驟傳遞數據而不進行修改
   - 可以與其他組件組合構建複雜數據管道
   - 對於需要保留原始輸入同時添加新字段的情況特別有幫助

3. **Input Handling**
   - 接受字典類型的輸入
   - 可以處理單個值
   - 在鏈式步驟中保持數據結構

In [9]:
from langchain_core.runnables import RunnablePassthrough

# Runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

這裏是 ```RunnablePassthrough``` 的一個例子。

In [10]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# The dict value has been updated with RunnablePassthrough().
runnable_chain.invoke(10)

AIMessage(content='10 times 10 is equal to 100.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dffc0a69-0ee5-43b1-adae-03ee863d5a68-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

這裏是使用 ```RunnablePassthrough.assign()``` 的結果比較。

In [11]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

```RunnablePassthrough.assign()```
- 結合 input 的 key/value 對和新增的 key/value 對。

In [12]:
# Input key: num, Assigned key: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

## Efficient Parallel Execution with RunnableParallel

```RunnableParallel``` 是一個用於同時執行多個 ```Runnable``` 物件的工具，它通過將輸入數據分配到不同的組件，收集其結果，並將其整合成一個統一的輸出。此功能使其成為一個強大的工具，用於優化工作流程，其中任務可以獨立和同時執行。


1. **Concurrent Execution**
   - 同時執行多個 ```Runnable``` 物件，減少需要進行平行化的任務所需時間。

2. **Unified Output Management**
   - 將所有平行執行的結果整合成一個單一、協調的輸出，簡化下游處理。

3. **Flexibility**
   - 可以處理多種輸入類型，並通過有效地分配工作負荷來支持複雜的工作流程。

In [ ]:
from langchain_core.runnables import RunnableParallel

# 創造 RunnableParallel 的實例。此實例允許多個 Runnable 物件同時執行。
runnable = RunnableParallel(
    # 將 RunnablePassthrough 實例作為 'passed' 關鍵字參數傳入。此功能僅將輸入數據通過而不進行修改。
    passed=RunnablePassthrough(),
    # 使用 RunnablePassthrough.assign 作為 'extra' 關鍵字參數分配 lambda 函數 'mult'。 
    # 這個函數將輸入字典中 'num' 金鑰關聯的值乘以 3。
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 將 lambda 函數作為 'modified' 關鍵字參數傳入。 
    # 這個函數將輸入字典中 'num' 金鑰關聯的值加 1。
    modified=lambda x: x["num"] + 1,
)

# 呼叫 runnable 的 invoke 方法，傳入一個字典 {'num': 1}。
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

Chains can also be applied to RunnableParallel.

In [14]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("What is the capital of {country}?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("What is the area of {country}?")
    | ChatOpenAI()
)

In [15]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("United States of America")

{'capital': AIMessage(content='The capital of the United States of America is Washington, D.C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-29437a26-8661-4f15-a655-3b3ca6aa0e8c-0', usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='The total land area of the United States of America is approximately 3.8 million square miles.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens

## Dynamic Processing with RunnableLambda

```RunnableLambda``` 是一個靈活的工具，允許開發者使用 lambda 函數定義自定義數據轉換邏輯。通過啟用快速且簡單的自定義處理工作流程的實現，```RunnableLambda``` 簡化了創建定制數據管道的過程，同時確保最小的設置開銷。

1. **Customizable Data Transformation**
   - 允許用戶使用 lambda 函數定義自定義邏輯來轉換輸入數據，提供無與倫比的靈活性。

2. **Lightweight and Simple**
   - 提供了一種簡單的方式來實現即時處理，而不需要定義繁雜的類或函數。


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    # Get today's date
    return datetime.today().strftime("%b-%d")

# Print today's date
get_today(None)

'Jan-04'

In [17]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Create the prompt and llm
prompt = PromptTemplate.from_template(
    "List {n} famous people whose birthday is on {today}. Include their date of birth."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create the chain
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
# Output
print(chain.invoke(3))

Here are three famous people born on January 4:

1. **Isaac Newton** - Born on January 4, 1643 (according to the Gregorian calendar; December 25, 1642, in the Julian calendar), he was an English mathematician, physicist, astronomer, and author who is widely recognized as one of the most influential scientists of all time.

2. **Louis Braille** - Born on January 4, 1809, he was a French educator and inventor of a system of reading and writing for use by the blind or visually impaired, known as Braille.

3. **Michael Stipe** - Born on January 4, 1960, he is an American singer-songwriter and the lead vocalist of the alternative rock band R.E.M.


## Extracting Specific Keys Using ```itemgetter```

```itemgetter``` 是一個從 Python 的 ```operator``` 模組中提取特定鍵或索引值的工具函數，具有以下特性和優點:

1. **Core Functionality**
   - 可以高效地從字典、元組和列表中提取特定鍵或索引值
   - 可以同時提取多個鍵或索引值
   - 支持函數式編程風格

2. **Performance Optimization**
   - 比常规索引更高效，特別是在重複鍵存取操作時
   - 優化內存使用
   - 在處理大量數據集時提供性能優勢

3. **Usage in LangChain**
   - 在鏈式組合中進行數據過濾
   - 從複雜的輸入結構中選擇性提取
   - 與其他 Runnable 物件組合用於數據預處理


In [19]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# Function that returns the length of a sentence.
def length_function(text):
    return len(text)


# Function that returns the product of the lengths of two sentences.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# Function that uses _multiple_length_function to return the product of the lengths of two sentences.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("What is {a} + {b}?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [20]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25 = 30', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1cb2a062-52ba-4042-a4c1-a1eef155f6cc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})